In [ ]:
import tensorflow as tf
import mann

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = tf.image.grayscale_to_rgb(tf.Variable(x_train.reshape(-1, 28, 28, 1)))
x_test = tf.image.grayscale_to_rgb(tf.Variable(x_test.reshape(-1, 28, 28, 1)))

In [ ]:
preprocess_model = tf.keras.models.Sequential()
preprocess_model.add(tf.keras.layers.Rescaling(1./255))
preprocess_model.add(tf.keras.layers.Resizing(40, 40, input_shape = (None, None, 3)))
preprocess_model.add(tf.keras.layers.RandomFlip('horizontal'))
preprocess_model.add(tf.keras.layers.RandomRotation(0.1))
preprocess_model.add(tf.keras.layers.RandomTranslation(0.1, 0.1))

true_model = tf.keras.models.Sequential()
true_model.add(mann.layers.MaskedConv2D(16, padding = 'same', input_shape = (40, 40, 3)))
true_model.add(mann.layers.MaskedConv2D(16, padding = 'same'))
true_model.add(tf.keras.layers.MaxPool2D())
true_model.add(tf.keras.layers.Dropout(0.2))
true_model.add(mann.layers.MaskedConv2D(32, padding = 'same', activation = 'relu'))
true_model.add(mann.layers.MaskedConv2D(32, padding = 'same', activation = 'relu'))
true_model.add(tf.keras.layers.MaxPool2D())
true_model.add(tf.keras.layers.Dropout(0.2))
true_model.add(mann.layers.MaskedConv2D(64, padding = 'same', activation = 'relu'))
true_model.add(mann.layers.MaskedConv2D(64, padding = 'same', activation = 'relu'))
true_model.add(tf.keras.layers.MaxPool2D())
true_model.add(tf.keras.layers.Dropout(0.2))
true_model.add(tf.keras.layers.Flatten())
true_model.add(mann.layers.MaskedDense(256, activation = 'relu'))
true_model.add(mann.layers.MaskedDense(256, activation = 'relu'))
true_model.add(mann.layers.MaskedDense(10, activation = 'softmax'))

training_input = tf.keras.layers.Input((None, None, 3))
training_x = preprocess_model(training_input)
training_output = true_model(training_x)
training_model = tf.keras.models.Model(
    training_input,
    training_output
)

training_model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

training_model.summary()

In [ ]:
callback = mann.utils.ActiveSparsification(
    0.85
)

training_model.fit(
    x_train,
    y_train,
    epochs = 200,
    batch_size = 512,
    validation_split = 0.2,
    callbacks = [callback]
)